###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Salud y Seguridad Social

In [13]:
import pandas as pd

###Archivo 4e4i-ua65.csv

Contiene información sobre los indicadores de mortalidad y morbilidad según departamento, municipio y año, generados por el Ministerio de Salud y Proteccion Social

In [14]:
df = pd.read_csv("https://www.datos.gov.co/resource/4e4i-ua65.csv?$limit=500000", dtype={'codmunicipio': str})
df.head()

,coddepartamento,departamento,codmunicipio,municipio,indicador,a_o,valor_indicador
0,5,Antioquia,05001,Medellín,MUERTE FETOINFANTIL,2005.0,29.652091
1,5,Antioquia,05001,Medellín,MUERTE FETOINFANTIL,2006.0,30.591086
2,5,Antioquia,05001,Medellín,MUERTE FETOINFANTIL,2007.0,28.257348
3,5,Antioquia,05001,Medellín,MUERTE FETOINFANTIL,2008.0,27.389642
4,5,Antioquia,05001,Medellín,MUERTE FETOINFANTIL,2009.0,25.566684


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266001 entries, 0 to 266000
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   coddepartamento  266001 non-null  int64  
 1   departamento     266001 non-null  object 
 2   codmunicipio     266001 non-null  object 
 3   municipio        266001 non-null  object 
 4   indicador        266000 non-null  object 
 5   a_o              265892 non-null  float64
 6   valor_indicador  266001 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 14.2+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [16]:
relevant_cols = ['departamento','codmunicipio','municipio' ,'indicador', 'a_o','valor_indicador' ]
mortality_and_morbidity_rates_minsalud = df[relevant_cols]
mortality_and_morbidity_rates_minsalud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266001 entries, 0 to 266000
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   departamento     266001 non-null  object 
 1   codmunicipio     266001 non-null  object 
 2   municipio        266001 non-null  object 
 3   indicador        266000 non-null  object 
 4   a_o              265892 non-null  float64
 5   valor_indicador  266001 non-null  float64
dtypes: float64(2), object(4)
memory usage: 12.2+ MB


### Verificar anos que cubre la informacion

In [17]:
mortality_and_morbidity_rates_minsalud['a_o'].unique()

array([2005., 2006., 2007., 2008., 2009., 2010., 2011., 2012., 2013.,
       2014., 2015., 2016., 2017., 2018., 2019., 2020.,   nan])

### Verificar valores nulos

In [18]:
mortality_and_morbidity_rates_minsalud.isnull().sum()

,0
departamento,0
codmunicipio,0
municipio,0
indicador,1
a_o,109
valor_indicador,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [19]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento','municipio' ,'indicador']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(mortality_and_morbidity_rates_minsalud[column].unique())
    print()


Categorías en la columna 'departamento':
['Antioquia' 'Atlántico' 'Bogotá, D.C.' 'Bolívar' 'Boyacá' 'Caldas'
 'Caquetá' 'Cauca' 'Cesar' 'Córdoba' 'Cundinamarca' 'Chocó' 'Huila'
 'La Guajira' 'Magdalena' 'Meta' 'Nariño' 'Norte de Santander' 'Quindio'
 'Risaralda' 'Santander' 'Sucre' 'Tolima' 'Valle del Cauca' 'Arauca'
 'Casanare' 'Putumayo' 'Amazonas'
 'Archipiélago de San Andrés, Providencia y Santa Catalina' 'Guainía'
 'Guaviare' 'Vaupés' 'Vichada']

Categorías en la columna 'municipio':
['Medellín' 'Amalfi' 'Abejorral' ... 'La Primavera' 'Santa Rosalía'
 'Cumaribo']

Categorías en la columna 'indicador':
['MUERTE FETOINFANTIL'
 'PORCENTAJE DE NACIDOS VIVOS A TÉRMINO CON BAJO PESO AL NACER'
 'PORCENTAJE DE NACIDOS VIVOS CON BAJO PESO AL NACER'
 'PORCENTAJE DE NACIDOS VIVOS CON CUATRO O MAS CONSULTAS DE CONTROL PRENATAL'
 'PORCENTAJE DE NACIDOS VIVOS DE MUJERES ENTRE 15 A 18 AÑOS  DONDE EL PADRE ES MAYOR 4 O MÁS AÑOS DE EDAD'
 'PORCENTAJE DE NACIDOS VIVOS DE MUJERES MENORES DE 15 AÑOS 

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [20]:
import unicodedata

# Función para eliminar acentos y caracteres especiales
def remove_accents_and_special_chars(input_str):
    if isinstance(input_str, str):  # Verifica si el valor es una cadena
        # Normalizar la cadena a NFKD
        nfkd_form = unicodedata.normalize('NFKD', input_str)

        # Eliminar acentos
        no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

        # Definir caracteres no deseados
        unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

        # Eliminar caracteres no deseados
        cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

        # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
        result = cleaned_str.strip().upper()

        return result
    else:
        # Si no es una cadena (e.g., NaN o None), devolver el valor tal cual
        return input_str

In [21]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    mortality_and_morbidity_rates_minsalud.loc[:, col] = mortality_and_morbidity_rates_minsalud[col].apply(remove_accents_and_special_chars)


- Codificación de algunas variables categóricas

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codmunicipio"

In [22]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codmunicipio" en el df mortality_and_morbidity_rates_minsalud

In [25]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
mortality_and_morbidity_rates_minsalud.loc[:,'codmunicipio'] = mortality_and_morbidity_rates_minsalud['codmunicipio'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = mortality_and_morbidity_rates_minsalud['codmunicipio'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [26]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [27]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,codmunicipio,count,percentage
0,5,266001,1.0


In [28]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = mortality_and_morbidity_rates_minsalud[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codmunicipio']])
    print()

Muestra de registros con longitud 5:
  codmunicipio
0        05001
1        05001
2        05001
3        05001
4        05001



Nota: el 100%  de los codigos que identifican a los municipios en el df mortality_and_morbidity_rates_minsalud corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [29]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [30]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

  - Comparar nombres de departamentos en los dataframes

In [31]:
# Comparar listas de departamento
compare_lists(mortality_and_morbidity_rates_minsalud['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en mortality_and_morbidity_rates_minsalud", "Departamentos dept_mpios_codes")

Departamentos en mortality_and_morbidity_rates_minsalud que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [33]:
# Comparar listas de códigos
compare_lists(mortality_and_morbidity_rates_minsalud['codmunicipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en mortality_and_morbidity_rates_minsalud", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en mortality_and_morbidity_rates_minsalud que no están en Códigos de municipios dept_mpios_codes:
{'63999', '85999', '27086', '27999', '94663', '19999', '94999', '05999'}


In [38]:
# Separar los registros con cod errado
codes_wrong = mortality_and_morbidity_rates_minsalud[mortality_and_morbidity_rates_minsalud['codmunicipio'].isin(['63999', '85999', '27086', '27999', '94663', '19999', '94999', '05999'])]
codes_wrong['codmunicipio'].value_counts()

,count
codmunicipio,
27086,58
94663,56
19999,9
05999,3
63999,3
27999,1
85999,1
94999,1


In [39]:
# Ver el primer registro de cada codigo errado
codes_wrong.groupby('codmunicipio').first()

,departamento,municipio,indicador,a_o,valor_indicador
codmunicipio,,,,,
05999,ANTIOQUIA,SIN INFORMACION - ANTIOQUIA,TASA DE MORTALIDAD FETAL,2011.0,1000.000000
19999,CAUCA,SIN INFORMACION - CAUCA,PORCENTAJE DE NACIDOS VIVOS A TERMINO CON BAJO...,2011.0,12.500000
27086,CHOCO,BELEN DE BAJIRA,MUERTE FETOINFANTIL,2005.0,9.950249
27999,CHOCO,SIN INFORMACION CHOCO,None,2015.0,0.071429
63999,QUINDIO,SIN INFORMACION - QUINDIO,TASA DE MORTALIDAD FETAL,2006.0,1000.000000
85999,CASANARE,SIN INFORMACION - CASANARE,PROMEDIO DE CONTROLES PRENATALES,2013.0,2.000000
94663,GUAINIA,MAPIRIPANA,MUERTE FETOINFANTIL,2018.0,166.666667
94999,GUAINIA,SIN INFORMACION - GUANIA,TASA DE MORTALIDAD FETAL,2005.0,1000.000000


Nota: Los registros que no tienen informacion del municipio los eliminamos porque no hay posibilidad de identificarlos. En la bd del proyecto Mapiripana en Guania, figura como vereda de Barrancaminas hacemos el ajuste, lo mismo los de Belen de Bajira que en la bd figura como vereda de Mutata.

In [40]:
# Eliminar filas donde 'municipio' contenga "SIN INFORMACION"
import re
mortality_and_morbidity_rates_minsalud = mortality_and_morbidity_rates_minsalud[~mortality_and_morbidity_rates_minsalud['municipio'].str.contains('SIN INFORMACION', case=False, regex=True)]


In [44]:
# Solucionar registros a nombre de MAPIRIPANA y BELEN DE BAJIRA
dept_mpios_codes[dept_mpios_codes['mupio_name'].isin(['MUTATA', 'BARRANCOMINAS'])]

,dept_mpio_code,dept_name,mupio_name
77,05480,ANTIOQUIA,MUTATA
1093,94343,GUAINIA,BARRANCOMINAS


In [49]:
# Reemplazamos cod de MAPIRIPANA y BELEN DE BAJIRA por los de BARRANCOMINAS y MUTATA
mortality_and_morbidity_rates_minsalud.loc[:, 'codmunicipio'] = mortality_and_morbidity_rates_minsalud['codmunicipio'].replace({'27086': '05480', '94663': '94343'})

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [50]:
# Comparar listas de códigos
compare_lists(mortality_and_morbidity_rates_minsalud['codmunicipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en mortality_and_morbidity_rates_minsalud", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en mortality_and_morbidity_rates_minsalud que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [52]:
# Eliminar columnas 'departamento', 'municipio' que ya no son necesarias
columns_to_drop = ['departamento', 'municipio']
final_mortality_and_morbidity_rates_minsalud = mortality_and_morbidity_rates_minsalud.drop(columns=columns_to_drop)

In [54]:
# Adicionar columna para trazabilidad de la fuente
final_mortality_and_morbidity_rates_minsalud['source_id'] = 114

In [53]:
final_mortality_and_morbidity_rates_minsalud.columns

Index(['codmunicipio', 'indicador', 'a_o', 'valor_indicador'], dtype='object')

In [55]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'codmunicipio': 'dane_code',
    'a_o': 'year',
    'indicador': 'indicator_name',
    'valor_indicador': 'indicator_values',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_mortality_and_morbidity_rates_minsalud.rename(columns=translation_map, inplace=True)

In [56]:
#Estructura final del dataset a integrar a la base de datos
final_mortality_and_morbidity_rates_minsalud.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265983 entries, 0 to 266000
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   dane_code         265983 non-null  object 
 1   indicator_name    265983 non-null  object 
 2   year              265874 non-null  float64
 3   indicator_values  265983 non-null  float64
 4   source_id         265983 non-null  int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 12.2+ MB


## Salvar en archivo csv en el drive

In [57]:
# Guardar en archivos CSV en el drive
final_mortality_and_morbidity_rates_minsalud.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/mortality_and_morbidity_rates_minsalud.csv', index=False)